## Iris Flower classification


In [13]:
from __future__ import absolute_import , division , print_function , unicode_literals

import tensorflow as tf
import pandas as pd

# Check TensorFlow version
print(tf.__version__)

# Create a simple tensor
# tensor = tf.constant([[1, 2], [3, 4]])
# print(tensor)
# tensor_reshape = tf.reshape(tensor,[4,1])
# print(tensor_reshape)


2.19.0


In [14]:
CSV_COLUMN_NAMES = ['SepalLength' , 'SepalWidth' , 'PetalLength' , 'PetalWidth' , 'Species']
SPECIES = ['Setosa'  , 'Versicolor' , 'Virginica']
#Constants to help us later on

In [23]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv" , "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
)
test_path = tf.keras.utils.get_file(
    "iris_test.csv" , "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)
# Load the datasets into pandas DataFrames
# Use header=0 to read the existing header, adjust if no header is present
train = pd.read_csv(train_path ,names =CSV_COLUMN_NAMES, header =0)
test = pd.read_csv(test_path ,names = CSV_COLUMN_NAMES, header = 0)
# Verify column names
print("Train columns:", train.columns.tolist())
print("Test columns:", test.columns.tolist())

Train columns: ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
Test columns: ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']


In [24]:
# Separate labels and convert to integer encoding if needed
train_y = train.pop('Species')
test_y = test.pop('Species')
# Convert features to numpy array
X_train = train.values
X_test = test.values

In [25]:
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)  # Compute mean and variance

## Tensor Flow data set

In [26]:


def create_dataset(features, labels, training=True, batch_size=256, num_epochs=None):
    """
    Creates a TensorFlow Dataset from features and labels.
    
    Args:
        features (dict): Dictionary of feature tensors or numpy arrays.
        labels: Label tensor or numpy array.
        training (bool): Whether to shuffle and repeat the dataset (default: True).
        batch_size (int): Size of each batch (default: 256).
        num_epochs (int, optional): Number of epochs to repeat the dataset (default: None, infinite if training).
    
    Returns:
        tf.data.Dataset: A batched and optionally shuffled/repeated dataset.
    """
    # Convert features dictionary and labels to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    
    # Shuffle and repeat if in training mode
    if training:
        dataset = dataset.shuffle(buffer_size=1000)  # Shuffle with a buffer of 1000 samples
        if num_epochs is not None:
            dataset = dataset.repeat(num_epochs)  # Repeat for specified epochs
        else:
            dataset = dataset.repeat()  # Infinite repeat if no epochs specified
    
    # Batch the dataset
    dataset = dataset.batch(batch_size)
    
    # Prefetch to improve performance
    dataset = dataset.prefetch(tf.data.AUTOTUNE)  # Allows background processing of data
    
    return dataset

## Training and Testing Datasets

In [27]:
# Create datasets
train_dataset = create_dataset(X_train, train_y, training=True, batch_size=32, num_epochs=10)
test_dataset = create_dataset(X_test, test_y, training=False, batch_size=32, num_epochs=1)

In [28]:
# Build the Keras model for multiclass classification
model = tf.keras.Sequential([
    normalizer,  # Apply normalization layer
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 units for 3 classes
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # For integer labels
              metrics=['accuracy'])



## Training and Evaluation

In [35]:
# Train the model
model.fit(train_dataset, epochs=10, steps_per_epoch=len(X_train) // 32)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset, steps=len(X_test) // 32)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7956 - loss: 0.5472
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8229 - loss: 0.5076
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8021 - loss: 0.5170
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7565 - loss: 0.5212
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7852 - loss: 0.5256
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7839 - loss: 0.5073   
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8203 - loss: 0.4768
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7513 - loss: 0.5244
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7995 - loss: 0.4879
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8190 - loss: 0.5073
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7000 - loss: 0.5995
Test Loss: 0.5995, Test Accuracy: 0.7000


## Predictions

In [36]:
# Optional: Make predictions
pred_probs = model.predict(test_dataset, steps=len(X_test) // 32)
pred_classes = tf.argmax(pred_probs, axis=1).numpy()

# Print some results
for i in range(min(10, len(test_y))):
    print(f"Sample {i+1}: Predicted = {pred_classes[i]}, Actual = {test_y.iloc[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Sample 1: Predicted = 2, Actual = 1
Sample 2: Predicted = 2, Actual = 2
Sample 3: Predicted = 0, Actual = 0
Sample 4: Predicted = 2, Actual = 1
Sample 5: Predicted = 1, Actual = 1
Sample 6: Predicted = 2, Actual = 1
Sample 7: Predicted = 0, Actual = 0
Sample 8: Predicted = 2, Actual = 2
Sample 9: Predicted = 2, Actual = 1
Sample 10: Predicted = 2, Actual = 2
